문제 : https://dacon.io/competitions/official/235689/overview/description/

출처 : https://dacon.io/codeshare/2400?page=1&dtype=recent

In [1]:
cd drive/MyDrive/dacon/운동 동작 분류 AI 경진대회

/content/drive/MyDrive/dacon/운동 동작 분류 AI 경진대회


In [2]:
ls

'1d cnn.ipynb'          model_kf/               train_features.csv
'1D CNN + LSTM.ipynb'   sample_submission.csv   train_labels.csv
 model_1dcnn+lstm/      test_features.csv


In [3]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Dropout, Flatten, BatchNormalization, Input, Convolution2D, Activation,TimeDistributed
from keras.layers import Input, multiply, concatenate, Activation, Masking, Reshape
from keras.layers import GlobalAveragePooling1D, Permute, Dropout
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob


train=pd.read_csv('train_features.csv')
train_labels=pd.read_csv('train_labels.csv')
test=pd.read_csv('test_features.csv')
submission=pd.read_csv('sample_submission.csv')

In [4]:
# 각 칼럼 표준화
column_list = ['acc_x','acc_y','acc_z','gy_x','gy_y','gy_z']
for column in column_list:
  train[column] = (train[column]-train[column].mean())/train[column].std()

In [5]:
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,2.307313,-0.029939,-0.432104,0.011776,-0.410858,-0.461007
1,0,1,2.421086,-0.063321,-0.504058,0.026484,-0.528052,-0.365459
2,0,2,2.444664,-0.056749,-0.654198,-0.037962,-0.596041,-0.366767
3,0,3,2.428612,-0.116782,-0.573407,0.066099,-0.725311,-0.401232
4,0,4,2.439474,-0.044220,-0.588885,0.091969,-0.784094,-0.408413
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.367431,-0.845648,0.503707,-0.461266,-1.413128,-1.092639
1874996,3124,596,-0.326315,-0.845833,0.578875,-0.474109,-1.382884,-1.097377
1874997,3124,597,-0.300794,-0.859728,0.652722,-0.436694,-1.341672,-1.136169
1874998,3124,598,-0.253120,-0.886873,0.673002,-0.367051,-1.346630,-1.148831


In [6]:
train_id, test_id = train_test_split(train['id'].unique(), test_size=0.01, random_state = 2)

In [7]:
# LSTM 모델에 넣기 위해서 transpose 해서 (600,6)으로 만들었습니다.
x_val = []
          
for uid in tqdm(test_id):
    temp = np.array(train[train['id'] == uid].iloc[:,2:], np.float32).T
    x_val.append(temp)

x_val = np.array(x_val, np.float32)
x_val = x_val[:,:,:,np.newaxis]
x_val_trans = np.transpose(x_val,(0,2,1,3))
x_val_trans.shape

100%|██████████| 32/32 [00:00<00:00, 231.21it/s]


(32, 600, 6, 1)

In [8]:
x_train = []

for uid in tqdm(train_id):
    temp = np.array(train[train['id'] == uid].iloc[:,2:], np.float32).T
    x_train.append(temp)

x_train = np.array(x_train, np.float32)
x_train = x_train[:,:,:,np.newaxis]
x_train.shape

100%|██████████| 3093/3093 [00:11<00:00, 274.00it/s]


(3093, 6, 600, 1)

In [9]:
train_arg = []
def aug(data, uid, shift = 0):
    shift_data = np.roll(data, shift, axis=2)
    train_arg.append(shift_data)
    
for _ in range(6):
    aug(x_train, 0, _*100)   

In [10]:
for i in range(5):
  train_arg[i] = np.transpose(train_arg[i],(0,2,1,3))

In [11]:
y = train_labels['label']
y_train = y[train_id]
y_test = y[test_id]

y_train = tf.keras.utils.to_categorical(y_train) 
y_test = tf.keras.utils.to_categorical(y_test)

In [12]:
x_arg = np.concatenate([train_arg[0] ,train_arg[1],train_arg[2] ,train_arg[3],train_arg[4]])
y_arg = np.concatenate([y_train,y_train,y_train, y_train, y_train])
x_arg = x_arg.reshape(-1,600,6)
x_arg.shape

(15465, 600, 6)

# model

In [13]:
def generate_model():
    ip = Input(shape=(600, 6))

    x = Masking()(ip)
    x = LSTM(units = 50, return_sequences = True)(x)
    x = LSTM(units=50)(x)
    x = Dropout(0.3)(x)

  
    y = Reshape((3600,1))(ip)
    y = Conv1D (kernel_size=3, filters=128, strides=3, padding='valid',kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    
    y = Conv1D (kernel_size=3, filters=128, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y)
   
    y = Conv1D (kernel_size=3, filters=128, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y)
    
    y = Conv1D (kernel_size=3, filters=256, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y) 

  
    y = Conv1D (kernel_size=3, filters=256, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y) 
     
    y = Conv1D (kernel_size=3, filters=256, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y) 
     
    y = Conv1D (kernel_size=3, filters=256, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y) 
   
    y = Dropout(0.5)(y)
    y = Flatten()(y)
    
    x = concatenate([x, y])

    out = Dense(61, activation='softmax')(x)

    model = Model(ip, out)
    #model.summary()
    return model
model = generate_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 600, 6)]     0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 3600, 1)      0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 1200, 128)    512         reshape[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 1200, 128)    512         conv1d[0][0]                     
______________________________________________________________________________________________

In [14]:
num_models=15
model_list=[]

for i in tqdm(range(num_models)):
    model = generate_model()
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    model.fit(x_arg,y_arg, epochs=25, batch_size=32,validation_split=0.2)
    model_list.append(model)
    model.save(f"./model_1dcnn+lstm/model_{i}.h5")

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch 1/25
387/387 [==============================] - 98s 224ms/step - loss: 2.2531 - accuracy: 0.5317 - val_loss: 1.1531 - val_accuracy: 0.6673
Epoch 2/25
387/387 [==============================] - 81s 209ms/step - loss: 0.9922 - accuracy: 0.7177 - val_loss: 0.7152 - val_accuracy: 0.7866
Epoch 3/25
387/387 [==============================] - 81s 209ms/step - loss: 0.6966 - accuracy: 0.7878 - val_loss: 0.5776 - val_accuracy: 0.8225
Epoch 4/25
387/387 [==============================] - 81s 209ms/step - loss: 0.5440 - accuracy: 0.8256 - val_loss: 0.5187 - val_accuracy: 0.8338
Epoch 5/25
387/387 [==============================] - 81s 208ms/step - loss: 0.4307 - accuracy: 0.8613 - val_loss: 0.4387 - val_accuracy: 0.8555
Epoch 6/25
387/387 [==============================] - 81s 209ms/step - loss: 0.3910 - accuracy: 0.8702 - val_loss: 0.4161 - val_accuracy: 0.8739
Epoch 7/25
387/387 [==============================] - 81s 209ms/step - loss: 0.3185 - accuracy: 0.8871 - val_loss: 0.3931 - val_ac

  7%|▋         | 1/15 [34:01<7:56:24, 2041.72s/it]

Epoch 1/25
387/387 [==============================] - 95s 220ms/step - loss: 2.3243 - accuracy: 0.5065 - val_loss: 1.0284 - val_accuracy: 0.7026
Epoch 2/25
387/387 [==============================] - 81s 208ms/step - loss: 1.0059 - accuracy: 0.7158 - val_loss: 0.7390 - val_accuracy: 0.7837
Epoch 3/25
387/387 [==============================] - 81s 209ms/step - loss: 0.7153 - accuracy: 0.7814 - val_loss: 0.6335 - val_accuracy: 0.8044
Epoch 4/25
387/387 [==============================] - 81s 209ms/step - loss: 0.5530 - accuracy: 0.8227 - val_loss: 0.4855 - val_accuracy: 0.8490
Epoch 5/25
387/387 [==============================] - 81s 209ms/step - loss: 0.4362 - accuracy: 0.8571 - val_loss: 0.4048 - val_accuracy: 0.8755
Epoch 6/25
387/387 [==============================] - 81s 208ms/step - loss: 0.3906 - accuracy: 0.8716 - val_loss: 0.3799 - val_accuracy: 0.8817
Epoch 7/25
387/387 [==============================] - 81s 208ms/step - loss: 0.3279 - accuracy: 0.8892 - val_loss: 0.3907 - val_ac

 13%|█▎        | 2/15 [1:07:57<7:21:57, 2039.82s/it]

Epoch 1/25
387/387 [==============================] - 94s 218ms/step - loss: 2.3090 - accuracy: 0.5106 - val_loss: 1.1193 - val_accuracy: 0.6725
Epoch 2/25
387/387 [==============================] - 80s 207ms/step - loss: 1.0205 - accuracy: 0.7060 - val_loss: 0.7121 - val_accuracy: 0.7824
Epoch 3/25
387/387 [==============================] - 80s 208ms/step - loss: 0.7176 - accuracy: 0.7861 - val_loss: 0.6932 - val_accuracy: 0.7947
Epoch 4/25
387/387 [==============================] - 80s 208ms/step - loss: 0.5728 - accuracy: 0.8157 - val_loss: 0.5882 - val_accuracy: 0.8099
Epoch 5/25
387/387 [==============================] - 80s 208ms/step - loss: 0.4907 - accuracy: 0.8392 - val_loss: 0.4771 - val_accuracy: 0.8613
Epoch 6/25
387/387 [==============================] - 80s 208ms/step - loss: 0.3919 - accuracy: 0.8651 - val_loss: 0.4515 - val_accuracy: 0.8655
Epoch 7/25
387/387 [==============================] - 80s 208ms/step - loss: 0.3360 - accuracy: 0.8907 - val_loss: 0.4093 - val_ac

KeyboardInterrupt: ignored

In [ ]:
models = []
for i in tqdm(range(0, 15)):
    model_name = f"model_{i}.h5"
    models.append(keras.models.load_model(model_name))

print(f"{len(models)} models reloaded")

In [ ]:
x_val_trans = x_val_trans.reshape(-1,600,6)
preds = np.zeros(shape=y_test.shape)
test_preds_list=[]

for model, i in zip(models, range(len(models))):
  b = model.predict(x_val_trans)
  print(f"Model {i+1}")
  preds = preds + b
  test_preds_list.append(b)
    
preds = preds / len(models)


In [ ]:
def logloss(y_true, y_pred, eps=1e-15):
    y_pred = np.clip(y_pred, eps, 1 - eps)
    return -(y_true * np.log(y_pred)).sum(axis=1).mean()

logloss(y_test, preds)

In [ ]:
for j in range(len(test_preds_list)):
  print('model {}'.format(j))
  print(logloss(y_test, test_preds_list[j]))
  # print('\n')

In [ ]:
test=pd.read_csv('test_features.csv')

column_list = ['acc_x','acc_y','acc_z','gy_x','gy_y','gy_z']
for column in column_list:
  test[column] = (test[column]-test[column].mean())/test[column].std()

x_sub = []

for uid in tqdm(test['id'].unique()):
    temp = np.array(test[test['id'] == uid].iloc[:,2:], np.float32).T
    x_sub.append(temp)

test_X = np.array(x_sub, np.float32)
test_X = test_X[:,:,:,np.newaxis]
test_X.shape

In [ ]:
test_X_trans = np.transpose(test_X,(0,2,1,3))

In [ ]:
preds = np.zeros(shape=(782,61))
test_preds_list=[]
for model, i in zip(models, range(len(models))):
    b = model.predict(test_X_trans)
    print(f"Model {i+1}")
    preds = preds + b
    test_preds_list.append(b)
  
preds = preds / len(models)

In [ ]:
submission.iloc[:,1:]=preds
submission.to_csv('final2_submission.csv', index=False)